In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

import copy

from pof.failure_mode import FailureMode
from interface.layouts import *

In [2]:
fm = FailureMode().set_default()

In [3]:
# Failure Mode Tests
mfml = make_failure_mode_layout(fm, prefix="fm-")

# Task Tests
task = fm.tasks['ocr']
mtl = make_task_layout(task)

mtf = make_task_form(task, prefix="")

# Trigger Tests
triggers= fm.tasks['ocr'].triggers
mttl = make_task_trigger_layout(triggers)

condition_triggers = fm.tasks['ocr'].triggers['condition']
#mctf = make_condition_trigger_form(condition_triggers)


# Impact Tests

impacts= fm.tasks['ocr'].impacts
mtil = make_task_impact_layout(impacts)

state_impacts = fm.tasks['ocr'].impacts['state']
msil = make_state_impact_layout(state_impacts)

condition_impacts = fm.tasks['ocr'].impacts['condition']
mcil = make_condition_impact_layout(condition_impacts)

condition_impact = fm.tasks['ocr'].impacts['condition']['wall_thickness']
mtif = make_condition_impact_form(condition_impact)

# Distribution Tests
failure_dist = fm.failure_dist
mdl = make_dist_layout(failure_dist)

In [5]:

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
app.layout = html.Div([
    html.Div(children='Test Output', id='test_output'),
    dcc.Graph(id="maintenance_strategy"),
    mfml
])

sep= '-'
collapse_ids = []
for fm_name, fm in {'fm': fm}.items():
    collapse_ids = collapse_ids + [fm_name + sep]
    for task in fm.tasks:
        collapse_ids = collapse_ids + [fm_name + sep + 'task' + sep + task + sep]

@app.callback(
    Output("test_output", "children"),
    #[Output(f"{prefix}collapse", "is_open") for prefix in collapse_ids],
    [Input(f"{prefix}collapse-button", "n_clicks") for prefix in collapse_ids],
    [State(f"{prefix}collapse", "is_open") for prefix in collapse_ids],
)
def toggle_collapses(*args):
    ctx = dash.callback_context

    state_id = ""
    collapse_id = ctx.triggered[0]['prop_id'].split('.')[0].replace('-collapse-button','')
    if collapse_id in collapses:

        state_id = collapse_id + '-collapse.is_open'
        ctx.states[state_id] = not ctx.states[state_id]

    is_open = tuple(ctx.states.values())

    return str("asdf") #is_open

ms_fig_update = fm.get_dash_ids(prefix ='fm-')



#Execute
app.run_server(mode='inline', debug=True)

In [31]:
"""@app.callback(
    #Output("maintenance_strategy", "figure"), 
    Output("test_output", "children"),
    [Input(dash_id,"value") for dash_id in ms_fig_update]
)
def update_maintenance_strategy(*args):

    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
    value = ctx.triggered[0]['value']

    #Temp
    dash_id = dash_id.replace('fm-', "")

    # Update the model
    #fm_local = copy.deepcopy(fm)
    fm_local =fm
    fm_local.dash_update(dash_id, value)

    fm_local.mc_timeline(t_end=200, n_iterations=100)
    df = fm_local.expected_cost_df()

    fig = px.area(
        df,
        x="time",
        y="cost_cumulative",
        color="task",
        title="Maintenance Strategy Costs",
    )

    #fig.update_yaxes(range=[0, graph_y_limit])

    return "asdf""""

([<State `fm-collapse.is_open`>,
  <State `fm-task-inspection-collapse.is_open`>,
  <State `fm-task-ocr-collapse.is_open`>,
  <State `fm-task-cm-collapse.is_open`>],)

In [29]:
mfml

InputGroup([InputGroupAddon(children=Checkbox(None), addon_type='prepend'), Button(children='fm', id='fm--collapse-button', color='link'), Col([Form(children=[Col(FormGroup(children=[Label(children='Alpha', className='mr-2'), Input(id='fm-failure_dist-alpha', type='number', value=50, debounce=True)], className='mr-3')), Col(FormGroup(children=[Label(children='Beta', className='mr-2'), Input(id='fm-failure_dist-beta', type='number', value=10, debounce=True)], className='mr-3')), Col(FormGroup(children=[Label(children='Gamma', className='mr-2'), Input(id='fm-failure_dist-gamma', type='number', value=10, debounce=True)], className='mr-3'))], inline=True), Collapse(children=Card(CardBody([InputGroup([InputGroupAddon(children=Checkbox(None), addon_type='prepend'), Button(children='inspection', id='fm-task-inspection-collapse-button', color='link'), Col([Form(children=[Col(FormGroup([Label(children='Probability Effective', html_for='p_effective'), Input(id='fm-task-inspection-p_effective', t

In [6]:
ms_fig_update

['fm-failure_dist-alpha',
 'fm-failure_dist-beta',
 'fm-failure_dist-gamma',
 'fm-task-inspection-p_effective',
 'fm-task-inspection-cost',
 'fm-task-inspection-consequence',
 'fm-task-inspection-trigger-condition-wall_thickness-lower',
 'fm-task-inspection-trigger-condition-wall_thickness-upper',
 'fm-task-inspection-trigger-state-initiation',
 'fm-task-inspection-impact-state-detection',
 'fm-task-ocr-p_effective',
 'fm-task-ocr-cost',
 'fm-task-ocr-consequence',
 'fm-task-ocr-trigger-condition-wall_thickness-lower',
 'fm-task-ocr-trigger-condition-wall_thickness-upper',
 'fm-task-ocr-trigger-state-detection',
 'fm-task-ocr-impact-condition-wall_thickness-target',
 'fm-task-ocr-impact-condition-wall_thickness-method',
 'fm-task-ocr-impact-condition-wall_thickness-axis',
 'fm-task-ocr-impact-state-initiation',
 'fm-task-ocr-impact-state-detection',
 'fm-task-ocr-impact-state-failure',
 'fm-task-cm-p_effective',
 'fm-task-cm-cost',
 'fm-task-cm-consequence',
 'fm-task-cm-trigger-state-